In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as  tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


# Data Loading (Text Data)

In [ ]:
# Load the data and labels
train_data = np.load("Data/numpy/alphabets/train_data.npy")
train_labels = np.load("Data/numpy/alphabets/train_labels.npy")

test_data = np.load("Data/numpy/alphabets/test_data.npy")
test_labels = np.load("Data/numpy/alphabets/test_labels.npy")

# Reshape data for CNN
train_data = train_data.reshape(-1, 21, 3, 1)
test_data = test_data.reshape(-1, 21, 3, 1)

# Convert labels to categorical format
num_classes = len(np.unique(train_labels))

train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

# Data Loading (Number Data)

In [ ]:
# Load the data and labels
train_data = np.load("Data/numpy/numbers/num_train_data.npy")
train_labels = np.load("Data/numpy/numbers/num_train_labels.npy")

test_data = np.load("Data/numpy/numbers/num_test_data.npy")
test_labels = np.load("Data/numpy/numbers/num_test_labels.npy")

# Reshape data for CNN
train_data = train_data.reshape(-1, 21, 3, 1)
test_data = test_data.reshape(-1, 21, 3, 1)

# Convert labels to categorical format
num_classes = len(np.unique(train_labels))

train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    train_data, train_labels, test_size=0.2, shuffle=True, stratify=train_labels
)

print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

# CNN Model: ASL Text Detection

In [ ]:
# Define the CNN model
model = Sequential(
    [
        Conv2D(256, kernel_size=(2, 1), activation="relu", input_shape=(21, 3, 1)),
        MaxPool2D((2, 1)),
        
        Conv2D(512, kernel_size=(2, 1), activation="relu"),
        MaxPool2D((2, 1)),
        Dropout(0.3),
        
        Flatten(),
        Dense(256, activation="relu"),
        Dropout(0.2),
        Dense(num_classes, activation="softmax"),
    ]
)

# CNN Model: ASL Number Detection

In [ ]:
# Define the CNN model
model = Sequential(
    [
        Conv2D(32, kernel_size=(2, 1), activation="relu", input_shape=(21, 3, 1)),
        MaxPool2D((2, 1)),
        
        Conv2D(64, kernel_size=(2, 1), activation="relu"),
        MaxPool2D((2, 1)),
        Dropout(0.3),
        
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.2),
        Dense(num_classes, activation="softmax"),
    ]
)

# Compilation and Training

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val), callbacks=[early_stopping])

In [ ]:
# Evaluate the model
score = model.evaluate(test_data, test_labels, verbose=0)
print(f'Test accuracy: {score[1] * 100:.2f}%')
print(f'Test accuracy: {score}')

In [ ]:
import matplotlib.pyplot as plt

epochs = [i for i in range(35)]
fig,ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(16,9)


ax[0].plot(epochs, train_acc, 'g', label='Training Accuracy')
ax[0].plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()


ax[1].plot(epochs, train_loss, 'g', label='Training Loss')
ax[1].plot(epochs, val_loss, 'r', label='Validation Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].set_title('Training & Validation Loss')
ax[1].legend()

# Saving Model

In [ ]:
# Save alphabet model
model.save('models/asl_alphabet_model.keras')

In [ ]:
# Save number model
model.save('models/asl_number_model.keras')

# Converting Keras Model into TensorFlow Lite model

In [ ]:
# load model
from tensorflow.keras.models import load_model

# model = load_model("models/asl_alphabet_model.keras")
model = load_model("models/asl_number_model_2.h5")

# Evaluate the model
score = model.evaluate(test_data, test_labels, verbose=0)
print(f'Test accuracy: {score[1] * 100:.2f}%')
print(f'Test accuracy: {score}')

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('models/asl_number_model.tflite', 'wb') as f:
  f.write(tflite_model)

# Converting .keras to .h5

In [ ]:
from tensorflow.keras.models import load_model

# model = load_model("models/asl_alphabet_model.keras")
model = load_model("models/asl_number_model.keras")
model.save('models/asl_number_model.h5')